In [88]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [89]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = "nHAF4VfPXSFW4yBs8w2MKnNty"
credentials['CONSUMER_SECRET'] = "ZnrJTcRKVq9pqrnLDFkeTZBrZ0z53av8cYwcLIXFzVM2xyFZHD"
credentials['ACCESS_TOKEN'] = "AAAAAAAAAAAAAAAAAAAAAPiIKwEAAAAA9WtZ8uEanLGM0HIGREA3wQNj9aU%3DG14tlXfigXCmX5uY3JR5t85Lgr1WYHcJmAQKXw8GO1QGQu80Dw"
#credentials['ACCESS_SECRET'] = ...

In [90]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [91]:
import tweepy #https://github.com/tweepy/tweepy
import csv

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.AppAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 3198:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
#     print(json.dumps(alltweets[5]._json, indent=2))
#     outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
    
    #write the csv  
#     with open(f'/Users/leosilva/development/thesis/TwitterDataMining/tweets/{screen_name}_tweets.csv', 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text"])
#         writer.writerows(outtweets)
    
#     pass

    return alltweets

In [92]:
import mysql.connector

def get_connection():
    connection = mysql.connector.connect(host = "localhost", user="root", passwd="root", db="TwitterDataMining")
    cursor = connection.cursor(buffered=True)
    return [connection, cursor]

In [99]:
import mysql.connector
from datetime import datetime


def store_tweets(alltweets):
    print("init of store tweets function")
    connection, cursor = get_connection()
    try:
        cursor.execute("SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        cursor.close()
        connection.close()

    id_user = 0
    user = None
    
    some_tweet = alltweets[0]
    
    id_str_user = some_tweet['user']['id_str']
    try:
        cursor.execute("SELECT id FROM User u WHERE u.id_str_twitter = '{}'".format(id_str_user))
        user = cursor.fetchone()
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        cursor.close()
        connection.close()

    if user is None:
        name = some_tweet['user']['name']
        screen_name = some_tweet['user']['screen_name']
        created_at = some_tweet['user']['created_at']
        date_time_obj = datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')
        url = some_tweet['user']['url']
        location = some_tweet['user']['location']
        try:
            cursor.execute("INSERT INTO User (id_str_twitter, name, screen_name, location, url, created_at) VALUES ('{}', '{}', '{}', '{}', '{}', '{}')".format(id_str_user, name, screen_name, location, url, date_time_obj))
            cursor.execute('SELECT last_insert_id()')
            id_user = int(cursor.fetchone()[0])
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            cursor.close()
            connection.close()
    else:
        id_user = int(user[0])

        
    for tweet in alltweets:
        id_str_tweet = tweet['id_str']
        try:
            cursor.execute("SELECT id FROM Tweet t WHERE t.id_str_twitter = '{}'".format(id_str_tweet))
            existing_tweet = cursor.fetchone()
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            cursor.close()
            connection.close()
            break
                       
        if existing_tweet is None:
            text = tweet['text'].replace("\'", "\"")
            created_at = tweet['created_at']
            date_time_obj = datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')
            favorite_count = int(tweet['favorite_count'])
            retweet_count = int(tweet['retweet_count'])
            lang = tweet['lang']
            try:
                cursor.execute("INSERT INTO Tweet (id_str_twitter, text, created_at, favorite_count, retweet_count, lang, id_user) VALUES ('{}', '{}', '{}', {}, {}, '{}', {})".format(id_str_tweet, text, date_time_obj, favorite_count, retweet_count, lang, id_user))
            except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)
                cursor.close()
                connection.close()
                break

    connection.commit()
    cursor.close()
    connection.close()
    print("end of store tweets function")

In [100]:
alltweets = get_all_tweets("adorilson")
store_tweets([tweet._json for tweet in alltweets])

init of store tweets function
end of store tweets function


In [101]:
def get_last_date(screen_name):
    connection, cursor = get_connection()

    cursor.execute("SELECT id FROM User u WHERE u.screen_name = '{}'".format(screen_name))
    id_user = cursor.fetchone()

    cursor.execute("SELECT created_at FROM Tweet t WHERE t.id_user = '{}' ORDER BY created_at".format(id_user[0]))
    last_date = cursor.fetchone()
    return last_date[0]

In [102]:
last_date = get_last_date("adorilson")
print(type(last_date))
base_date = datetime(2017, 1, 1)
print(type(base_date))
if last_date > base_date:
    print("last_date maior")

<class 'datetime.datetime'>
<class 'datetime.datetime'>
last_date maior


In [103]:
import json

ids_twitter = []
f = open("/Users/leosilva/development/thesis/TwitterScraper/json/adorilson.json")
json_tweets = json.load(f)
for tweet in json_tweets:    
    ids_twitter.append(json_tweets[tweet]["id_str"])

In [104]:
def get_tweets_by_id_twitter(ids_twitter):
    ids_twitter_tuple = tuple(ids_twitter)
    connection, cursor = get_connection()

    cursor.execute("SELECT t.id_str_twitter FROM Tweet t WHERE t.id_str_twitter IN {}".format(ids_twitter_tuple))
    ids = cursor.fetchmany()

    return ids

In [105]:
existing_ids = get_tweets_by_id_twitter(ids_twitter)
print(existing_ids[0])

('964515079466180608',)


In [106]:
tweets_to_insert = [json_tweets[tweet] for tweet in json_tweets if json_tweets[tweet]["id_str"] not in existing_ids[0]]

In [107]:
store_tweets(tweets_to_insert)

init of store tweets function


KeyError: 'text'